In [1]:
import csv
def store(data):
    with open('CutData/Pseudo/NTF_200/Features_NTF2_1/Emot_NTF2_1.csv', 'a', newline='') as csvfile:
        fieldnames = ['Emot(num)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for line in data:
            writer.writerow({'Emot(num)': str(line)})

In [2]:
import csv
def score_store(data):
    #print(data)
    with open('CutData/Pseudo/NTF_200/Features_NTF2_1/EmotMean_NTF2_1.csv', 'a', newline='') as csvfile:
        fieldnames = ['EmotMean(score)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for line in data:
            writer.writerow({'EmotMean(score)': str(line)})   
#     with open('test.csv', 'a', newline='') as csvfile:
#         writer = csv.writer(csvfile, delimiter=',')
#         csvfile.write('貼文表情符號意義情感分數加總'+'\n')
#         for line in data: 
#              writer.writerow(str(line))

In [3]:
import os, sys
import warnings
warnings.filterwarnings('ignore')

In [4]:
import xml.etree.ElementTree as ET
import html.parser
from xml.sax.saxutils import unescape
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
import emot
stop_words = set(stopwords.words('english')) 
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','<','br','>','Q','A'])
doc = open('CutData/Pseudo/NTF_200/Features_NTF2_1/Doc_NTF2_1.csv', 'r').read() 
doc_num = doc.split('\n')  
tree = ET.parse('Formspring/new_XMLMergedFile.xml')
root = tree.getroot()

In [5]:
total_sentiscore_list=[]
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        html_parser = html.parser.HTMLParser()
        content = html_parser.unescape(child.text)
        content=content.replace('<br>',' ')
        ans = emot.emoticons(content)
        try:
            if ans['mean'] !=[]:
                #print(ans['mean'])
                emot_content=str(ans['mean']).strip('[]').strip("'")
                stokens = nltk.word_tokenize(emot_content)#斷詞
                filtered_sentence = [w for w in stokens if not w in stop_words]#刪除停用詞
                taggedlist=nltk.pos_tag(filtered_sentence)#詞性標註
                wnl = nltk.WordNetLemmatizer() #做詞性還原
                score_list = []
                for t in taggedlist:
                    lemmatized = wnl.lemmatize(t[0])
                    newtag=''
                    if t[1].startswith('NN'):
                        newtag='n'
                    elif t[1].startswith('JJ'):
                        newtag='a'
                    elif t[1].startswith('VB'):
                        newtag='v'
                    elif t[1].startswith('RB'):
                        newtag='r'
                    else:
                        newtag=''       
                    if(newtag != ''):
                        synsets = list(swn.senti_synsets(lemmatized, newtag))
                        score = 0
                        if(len(synsets) > 0):
                            nosenti_count = 0
                            for syn in synsets:
                                if syn.pos_score() == 0 and syn.neg_score() == 0:
                                    nosenti_count += 1
                                    continue
                                else:
                                    score +=syn.pos_score()-syn.neg_score()       
                            if len(synsets) != nosenti_count:
                                score_list.append(score/(len(synsets)-nosenti_count)) 
                        else:
                            continue
                if score_list != []:
                    total_sentiscore = 0
                    for score in score_list:
                        total_sentiscore += score  
                    #print(total_sentiscore)
                    total_sentiscore_list.append(total_sentiscore)
                else:
                    total_sentiscore_list.append(0)
            else:
                 #print('0')
                 total_sentiscore_list.append(0)
        except:
            total_sentiscore_list.append(0)
score_store(total_sentiscore_list)

In [6]:
length_emot=[]
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        html_parser = html.parser.HTMLParser()
        content = html_parser.unescape(child.text)
        content=content.replace('<br>',' ')
        ans = emot.emoticons(content)
        try:
            #length_emot.append(len(ans['value'])) 
            length_emot.append(len(ans['value']))
        except:
            length_emot.append(0)
store(length_emot)